In [49]:
import gym
env = gym.make('Copy-v0')

m = {}

print('Observation space:', env.observation_space)
print('Action space:', env.action_space)
print('---')
print()

"""
idx: 0
    0: flyt hoved venstre
    1: flyt hoved højre
idx: 1
    0: læs
    1: skriv
idx: 2
    0: skriv 'A'
    1: skriv 'B'
    2: skriv 'C'
    3: skriv 'D'
    4: skriv 'E'
"""

"""
Algorithm:
læs og flyt til venstre indtil observation = 5
læs og flyt til højre
hvis 5: done
ellers: skriv obs
"""
STEP_LEFT_READ = (0,0,0)
STEP_RIGHT_READ = (1,0,0)
STEP_RIGHT_WRITE = (1,1,)

def act_render(env, action):
    observation, reward, done, info = env.step(action)
    env.render()
    return observation, reward, done, info

env.reset()
env.render()
print('BEGIN')
print()
observation, reward, done, info = act_render(env, STEP_LEFT_READ)
while observation < 5: 
    observation, reward, done, info = act_render(env, STEP_LEFT_READ)
observation, reward, done, info = act_render(env, STEP_RIGHT_READ)
while not done:
    observation, reward, done, info = act_render(env, STEP_RIGHT_WRITE + (observation,))

print('END')
    
env.close()

Observation space: Discrete(6)
Action space: Tuple(Discrete(2), Discrete(2), Discrete(5))
---

Total length of input instance: 4, step: 0
Observation Tape    :   DAAE  
Output Tape         :   
Targets             :   DAAE  






BEGIN

Total length of input instance: 4, step: 1
Observation Tape    :   DAAE  
Output Tape         :   
Targets             :   DAAE  

Current reward      :   0.000
Cumulative reward   :   0.000
Action              :   Tuple(move over input: left,
                              write to the output tape: False,
                              prediction: A)
Total length of input instance: 4, step: 2
Observation Tape    :   DAAE  
Output Tape         :   
Targets             :   DAAE  

Current reward      :   0.000
Cumulative reward   :   0.000
Action              :   Tuple(move over input: right,
                              write to the output tape: False,
                              prediction: A)
Total length of input instance: 4, step: 3
Observation Ta

In [13]:
import gym
env = gym.make('CartPole-v0')
env.reset()

for i in range(1000):
    observation, reward, done, info = env.step(env.action_space.sample())
    env.render()
    if done: break
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
